# Necessary Function

In [ ]:
!pip install gensim

In [ ]:
import pandas as pd
# nltk
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk.corpus
nltk.download('stopwords')
from nltk.stem.porter import *

# feature extraction
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from textblob import TextBlob

# gensim
import gensim
from gensim import corpora, models, similarities, matutils

# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB


import sklearn.metrics as mt
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix

from sklearn.preprocessing import StandardScaler, MinMaxScaler,MaxAbsScaler
import string
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import emoji
import re

def strip_emoji(text):

    print(emoji.emoji_count(text))

    new_text = re.sub(emoji.get_emoji_regexp(), r"", text)

    return new_text

In [ ]:
df = pd.read_csv("tripadvisor_hotel_review_combined.csv") 

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
# Number of missing values in the dataset
df.isna().sum()

In [ ]:
df.Rating.value_counts()

# Clean Data

In [ ]:
def remove_url(txt):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',txt)

def remove_html(txt):
    html=re.compile(r'<.*?>')
    return html.sub(r'',txt)

# U+1F970
def remove_emoji(txt):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', txt)
def remove(emoji):
    em = re.compile(r"🥰")
    return em.sub(r"",emoji)
def remove_blank_space(text):
    return re.sub("\s\s+" , " ", text)
def remove_blank2(text):
    text = text.strip()
    return text
def remove_all(ReviewText):
    ReviewText = ReviewText.replace("(<br/>)", "")
    ReviewText = ReviewText.replace('(<a).*(>).*(</a>)', '')
    ReviewText = ReviewText.replace('(&amp)', '')
    ReviewText = ReviewText.replace('(&gt)', '')
    ReviewText = ReviewText.replace('(&lt)', '')
    ReviewText = ReviewText.replace('(\xa0)', ' ') 
    ReviewText = ReviewText.replace(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', ' ') 
    return ReviewText

from wordcloud import STOPWORDS
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

s_words= list(stopwords.words('english'))
stop_words = list(STOPWORDS)+ ["what", "us", "this","well","there","much","us","and","you're","in","where","when","just","how","is","ha","re","are"
                              "hi","aren't", 'couldn','could','couldnt',"couldn't",'did','had','have','must','does','should','was',"it's"
                               "didn't", "doesn't", "don't", "hadn't", "hasn't", "haven't", "isn't", 'let', 'll',"may",'were','is','has','must',
                               'mustn', 'rt', 'shan', 'shouldn', 'wasn', 'weren', 'won', 'wouldn','realli','now','got','man','people','a',
                               'becaus','caus',"one","im","guy","someone","two","nearby","i","he's","she's","we","it","they","wouldn’t","i've",
                               'aren', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'sdidn', 've',"will","restaurant"]



def remove_stopwords(txt):
    
    sentence = []
    s = ""
    for word in txt.split():    
        if(word not in stop_words):      
            sentence.append(word)
            s = ' '.join(sentence)
    return s

In [ ]:
df.Review = df.Review.apply(remove_url)
df.Review = df.Review.apply(remove_html)
df.Review = df.Review.apply(remove_emoji)
df.Review = df.Review.apply(remove)
df.Review = df.Review.apply(remove_blank_space)
df.Review = df.Review.apply(remove_blank2)
df.Review = df.Review.apply(remove_all)
df.Review = df.Review.map(remove_stopwords)
df.Review

In [ ]:
sentiment_data = df[['Rating','Review','Hotel_Name']]

In [ ]:
sentiment_data.values

In [ ]:
sdata = sentiment_data.values    # Gives a numpy array
count = len(sdata)
all_data = []

for i in range(count):
    rating = sdata[i, 0]
    hotel_name = sdata[i, 2]
    reviews = sdata[i, 1].split('], [')[0]     # Splitting the reviews and date strings from a single list and considering only the reviews
    reviews = reviews.replace("[[", "")
    reviews = reviews.replace("'", "")
    reviews = reviews.replace('"', '')
    reviews = reviews.split(',')
    print(reviews)
    for review in reviews:
        all_data.append([review, rating, hotel_name])

In [ ]:
all_data

In [ ]:
sent_data = pd.DataFrame(all_data, columns = ['Review', 'Rating','Hotel_Name'])

In [ ]:
sent_data

In [ ]:
# Tokenization
# Splitting sentences into list of individual words  
tokenized_data = sent_data['Review'].apply(lambda x : x.lower().split())    
tokenized_data

In [ ]:
# Stemming
# Reducing a word to its stem word
stemmer = PorterStemmer()
stem_data = tokenized_data.apply(lambda x: [stemmer.stem(i) for i in x])  
stem_data

In [ ]:
# Joining the stemmed words to reframe sentences
stemmed_data = []
for i in range(len(stem_data)):
    stemmed_data.append(' '.join(stem_data[i]))    

stemmed_data

In [ ]:
np.array(stemmed_data).reshape(-1,1)   # Converting into a numpy array

In [ ]:
sent_data['Cleaned_Review'] = np.array(stemmed_data).reshape(-1,1)  # Reshaping

# Visualizing customer reviews using WordCloud module

In [ ]:
# ALl the stemmed words in reviews 
all_words = ' '.join([text for text in sent_data['Cleaned_Review']])    #stemmed words

from wordcloud import WordCloud
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words)

plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
# Words used by the customers in reviews corresponding to rating 5
# Rating the restaurants as excellent
hide_words = ' '.join([text for text in sent_data['Cleaned_Review'][sent_data['Hotel_Name'] == 'HIDE BANGKOK HOSTEL']])

wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(hide_words)
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
# Words used by the customers in reviews corresponding to rating 5
# Rating the restaurants as excellent
excellent_words = ' '.join([text for text in sent_data['Cleaned_Review'][sent_data['Rating'] == 50]])

wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(excellent_words)
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

# Sentiment Analysis

In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

listy = []

for index, row in sent_data.iterrows():
  ss = sid.polarity_scores(row["Cleaned_Review"])
  listy.append(ss)
  
se = pd.Series(listy)
sent_data['polarity'] = se.values

display(sent_data.head(100))

In [ ]:
sent_data.to_csv('Trip_advisor_sentiment.csv', encoding='utf-8')